## MongoDB connection

In [15]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Create or connect to a database and drop it
client.drop_database("spotify_database")

# Create or connect to a database
db = client["spotify_database"]

# Create or connect to a collection
collection = db["track_data"]

print("Connected to MongoDB!")

Connected to MongoDB!


In [16]:
# Load JSON data from the file
import json

with open("./data/track_data.json", "r") as json_file:
    track_data = json.load(json_file)

# Check the first record (optional)
print(track_data[0])

{'track_id': '0027zf5idrflGmhXsvdVJr', 'track_name': 'She Calls Me Back', 'artist': {'artist_id': '2RQXRUsr4IW1f3mKyKsy4B', 'artist_name': 'Noah Kahan'}, 'album': {'album_id': '3qBWNcWifNhUKJAWzswdJY', 'album_name': 'Stick Season (Forever)', 'release_date': '2024-02-09', 'total_tracks': 30}, 'duration_ms': 243815, 'isrc': 'USUM72217169', 'highest_popularity': 56, 'audio_features': {'danceability': 0.749, 'energy': 0.687, 'key': 11.0, 'loudness': -6.497, 'mode': 1.0, 'speechiness': 0.0364, 'acousticness': 0.0786, 'instrumentalness': 0.0, 'liveness': 0.28, 'valence': 0.723, 'tempo': 127.905, 'time_signature': 4.0}, 'listening_history': [{'played_at': '2024-10-22T17:29:19.305Z', 'context_type': 'playlist', 'popularity': 55}, {'played_at': '2024-10-16T16:13:27.872Z', 'context_type': 'playlist', 'popularity': 55}, {'played_at': '2024-10-06T13:21:37.674Z', 'context_type': 'playlist', 'popularity': 56}]}


In [17]:
# Insert data into the MongoDB collection
collection.insert_many(track_data)

print("Data successfully inserted into MongoDB!")

Data successfully inserted into MongoDB!


In [18]:
# Query the first document in the collection
first_document = collection.find_one()
print(first_document)

# Count the total number of documents in the collection
print(f"Total documents in collection: {collection.count_documents({})}")

{'_id': ObjectId('67464b9cc8055717f1bdeb7e'), 'track_id': '0027zf5idrflGmhXsvdVJr', 'track_name': 'She Calls Me Back', 'artist': {'artist_id': '2RQXRUsr4IW1f3mKyKsy4B', 'artist_name': 'Noah Kahan'}, 'album': {'album_id': '3qBWNcWifNhUKJAWzswdJY', 'album_name': 'Stick Season (Forever)', 'release_date': '2024-02-09', 'total_tracks': 30}, 'duration_ms': 243815, 'isrc': 'USUM72217169', 'highest_popularity': 56, 'audio_features': {'danceability': 0.749, 'energy': 0.687, 'key': 11.0, 'loudness': -6.497, 'mode': 1.0, 'speechiness': 0.0364, 'acousticness': 0.0786, 'instrumentalness': 0.0, 'liveness': 0.28, 'valence': 0.723, 'tempo': 127.905, 'time_signature': 4.0}, 'listening_history': [{'played_at': '2024-10-22T17:29:19.305Z', 'context_type': 'playlist', 'popularity': 55}, {'played_at': '2024-10-16T16:13:27.872Z', 'context_type': 'playlist', 'popularity': 55}, {'played_at': '2024-10-06T13:21:37.674Z', 'context_type': 'playlist', 'popularity': 56}]}
Total documents in collection: 1021


In [19]:
# Update each document with total_listening_count
for track in collection.find():
    total_count = len(track["listening_history"])
    collection.update_one(
        {"_id": track["_id"]},
        {"$set": {"total_listening_count": total_count}}
    )

print("Added total_listening_count to all documents!")

Added total_listening_count to all documents!


In [20]:
from datetime import datetime, timedelta

# Function to adjust UTC to local time
def adjust_to_local_time(utc_time_str):
    try:
        utc_time = datetime.strptime(utc_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    except ValueError:
        utc_time = datetime.strptime(utc_time_str, "%Y-%m-%dT%H:%M:%SZ")
    local_time = utc_time - timedelta(hours=6)  # Nashville Standard Time (UTC-6)
    return local_time.strftime("%Y-%m-%dT%H:%M:%S")

# Update each document in MongoDB
for track in collection.find():
    for history in track["listening_history"]:
        # Replace 'played_at' with local time
        history["played_at"] = adjust_to_local_time(history["played_at"])
    
    # Update the document in MongoDB
    collection.update_one(
        {"_id": track["_id"]},
        {"$set": {"listening_history": track["listening_history"]}}
    )

print("All timestamps replaced with local time and updated in MongoDB!")

All timestamps replaced with local time and updated in MongoDB!


### 1) Top 10 Songs I Listened to the Most

In [21]:
# Find the top 10 songs with the highest total_listening_count
top_10_most_listened_songs = collection.find(
    {},
    {"track_name": 1, "total_listening_count": 1, "_id": 0}
).sort("total_listening_count", -1).limit(10)

print("Your Top 10 Most Listened Songs:")
for i, song in enumerate(top_10_most_listened_songs, 1):
    print(f"{i}. {song['track_name']} - {song['total_listening_count']} listens")

Your Top 10 Most Listened Songs:
1. Pink Pony Club - 28 listens
2. Good Luck, Babe! - 21 listens
3. That’s So True - 18 listens
4. HOT TO GO! - 18 listens
5. BIRDS OF A FEATHER - 15 listens
6. Sailor Song - 14 listens
7. Tough Love - 14 listens
8. Slow It Down - 14 listens
9. Hallucinogenics - 14 listens
10. I Can Do It With a Broken Heart - 14 listens


### 2) Top 10 Songs I Listened to the Most During the Day (9am to 5pm)

In [22]:
pipeline = [
    {"$unwind": "$listening_history"},
    {"$addFields": {
        "hour": {"$hour": {"$dateFromString": {"dateString": "$listening_history.played_at"}}}
    }},
    {"$match": {"hour": {"$gte": 9, "$lt": 17}}},
    {"$group": {"_id": "$track_name", "day_count": {"$sum": 1}}},
    {"$sort": {"day_count": -1}},
    {"$limit": 10}
]

top_10_day_songs = list(collection.aggregate(pipeline))

print("Your Top 10 Most Listened Songs During the Day:")
for i, song in enumerate(top_10_day_songs, 1):
    print(f"{i}. {song['_id']} - {song['day_count']} listens during the day")

Your Top 10 Most Listened Songs During the Day:
1. Pink Pony Club - 13 listens during the day
2. I Lied (with Allison Ponthier) - 12 listens during the day
3. Close To You - 12 listens during the day
4. HOT TO GO! - 11 listens during the day
5. i am not who i was - 10 listens during the day
6. Please Please Please - 10 listens during the day
7. THINGS BEHIND THINGS BEHIND THINGS - 10 listens during the day
8. Good Luck, Babe! - 9 listens during the day
9. Hallucinogenics - 8 listens during the day
10. us. (feat. Taylor Swift) - 8 listens during the day


### 3) Top 10 Songs I Listened to the Most During the Night (5pm to 9am)

In [23]:
pipeline = [
    {"$unwind": "$listening_history"},
    {"$addFields": {
        "hour": {"$hour": {"$dateFromString": {"dateString": "$listening_history.played_at"}}}
    }},
    {"$match": {"$or": [{"hour": {"$gte": 17}}, {"hour": {"$lt": 9}}]}},
    {"$group": {"_id": "$track_name", "night_count": {"$sum": 1}}},
    {"$sort": {"night_count": -1}},
    {"$limit": 10}
]

top_10_night_songs = list(collection.aggregate(pipeline))

print("Your Top 10 Most Listened Songs During the Night:")
for i, song in enumerate(top_10_night_songs, 1):
    print(f"{i}. {song['_id']} - {song['night_count']} listens during the night")

Your Top 10 Most Listened Songs During the Night:
1. Pink Pony Club - 15 listens during the night
2. Close To You - 14 listens during the night
3. Good Luck, Babe! - 12 listens during the night
4. I Can Do It With a Broken Heart - 11 listens during the night
5. That’s So True - 11 listens during the night
6. Slow It Down - 10 listens during the night
7. Sailor Song - 10 listens during the night
8. S P E Y S I D E - 9 listens during the night
9. BIRDS OF A FEATHER - 9 listens during the night
10. Linger - SiriusXM Session - 9 listens during the night


### 4) Top 10 Artists I Listened to the Most

In [24]:
pipeline = [
    {"$group": {"_id": "$artist.artist_name", "total_count": {"$sum": "$total_listening_count"}}},
    {"$sort": {"total_count": -1}},
    {"$limit": 10}
]

top_10_artists = list(collection.aggregate(pipeline))

print("Your Top 10 Most Listened Artists:")
for i, artist in enumerate(top_10_artists, 1):
    print(f"{i}. {artist['_id']} - {artist['total_count']} listens")

Your Top 10 Most Listened Artists:
1. Lord Huron - 170 listens
2. Matt Maeson - 165 listens
3. Gracie Abrams - 159 listens
4. Chappell Roan - 157 listens
5. Taylor Swift - 87 listens
6. Noah Kahan - 83 listens
7. Sabrina Carpenter - 61 listens
8. Bon Iver - 55 listens
9. Nothing But Thieves - 44 listens
10. The 1975 - 39 listens


### 5) Top 10 Albums I Listened to the Most

In [25]:
pipeline = [
    {"$group": {"_id": "$album.album_name", "total_count": {"$sum": "$total_listening_count"}}},
    {"$sort": {"total_count": -1}},
    {"$limit": 10}
]

top_10_albums = list(collection.aggregate(pipeline))

print("Your Top 10 Most Listened Albums:")
for i, album in enumerate(top_10_albums, 1):
    print(f"{i}. {album['_id']} - {album['total_count']} listens")

Your Top 10 Most Listened Albums:
1. The Rise and Fall of a Midwest Princess - 126 listens
2. The Secret of Us - 79 listens
3. Bank On The Funeral - 68 listens
4. The Secret of Us (Deluxe) - 48 listens
5. Short n' Sweet - 43 listens
6. Strange Trails - 43 listens
7. Lonesome Dreams (Bonus Track Version) - 36 listens
8. THE TORTURED POETS DEPARTMENT - 32 listens
9. Long Lost - 28 listens
10. Stick Season - 28 listens


### 6) Top 10 most popular songs I listened to

In [26]:
pipeline = [
    {"$sort": {"highest_popularity": -1}},
    {"$limit": 10},
    {"$project": {"track_name": 1, "highest_popularity": 1}}
]

top_10_popular_songs = list(collection.aggregate(pipeline))

if top_10_popular_songs:
    print("Your top 10 most popular songs:")
    for i, song in enumerate(top_10_popular_songs, 1):
        print(f"{i}. {song['track_name']} (Popularity: {song['highest_popularity']})")
else:
    print("No popular songs data available.")

Your top 10 most popular songs:
1. Die With A Smile (Popularity: 100)
2. BIRDS OF A FEATHER (Popularity: 99)
3. APT. (Popularity: 97)
4. Good Luck, Babe! (Popularity: 97)
5. Please Please Please (Popularity: 96)
6. Taste (Popularity: 96)
7. Timeless (with Playboi Carti) (Popularity: 92)
8. Who (Popularity: 92)
9. Bed Chem (Popularity: 92)
10. Sailor Song (Popularity: 92)


### 7) Top 10 least popular songs I listened to

In [27]:
pipeline = [
    {"$sort": {"highest_popularity": 1}},
    {"$limit": 10},
    {"$project": {"track_name": 1, "highest_popularity": 1}}
]

top_10_least_popular_songs = list(collection.aggregate(pipeline))

if top_10_least_popular_songs:
    print("Your top 10 least popular songs:")
    for i, song in enumerate(top_10_least_popular_songs, 1):
        print(f"{i}. {song['track_name']} (Popularity: {song['highest_popularity']})")
else:
    print("No least popular songs data available.")

Your top 10 least popular songs:
1. Tough Love (Popularity: 0)
2. Fabulous (Popularity: 0)
3. Risk (Popularity: 0)
4. Let It Happen (Popularity: 0)
5. Felt Good About You (Popularity: 0)
6. Holidays (Popularity: 0)
7. Gave You I Gave You I (Popularity: 0)
8. I Knew It, I Know You - Live From Vevo (Popularity: 0)
9. Still the One (Popularity: 0)
10. BAD DREAM (Popularity: 0)


### 8) Top 5 Listening Hours

In [ ]:
pipeline = [
    {"$unwind": "$listening_history"},
    {"$addFields": {"hour": {"$hour": {"$dateFromString": {"dateString": "$listening_history.played_at"}}}}},
    {"$group": {"_id": "$hour","listening_count": {"$sum": 1}}},
    {"$sort": {"listening_count": -1}},
    {"$limit": 5}
]

# Execute the query
top_listening_hours = list(collection.aggregate(pipeline))

# Display the results in a human-readable format
print("Top 5 Hours You Listen to Music:")
for i, entry in enumerate(top_listening_hours, 1):
    print(f"{i}. Hour {entry['_id']}: {entry['listening_count']} listens")

Top 5 Hours You Listen to Music:
1. Hour 12: 250 listens
2. Hour 22: 223 listens
3. Hour 13: 213 listens
4. Hour 23: 190 listens
5. Hour 15: 181 listens
